## CTIPS
* https://ctips-prod.dot.ca.gov/ctips/LoginMediatorForm.do

### To do
* Get total cost
    * total_cost: The total cost of this project.
    Total project cost can be calculated using 3 tables: project, fundtype, and fundline
    AH: which columns do I use from fundtype, project, and fundline to calculate the total cost?
    You can calculate total programmed for a project using: fundline.action = P and project.high_offlc = 1
    Then sum ( fundline.pe_paed + fundline.pe_env + fundline.pe_rw + fundline.pe_con + fundline.rw + fundline.con )

* Ask if DSHOPP means draft SHOPP project
* PROJSCHE - not a lot of matches

In [ ]:
import pandas as pd 
import sqlalchemy 
import sys 
import re
import oracledb 
import _utils
import _csis_utils

In [ ]:
oracledb.version = "8.3.0" 
sys.modules["cx_Oracle"] = oracledb 

In [ ]:
pd.options.display.max_columns = 400
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:

ENGINE_PATH_WIN_AUTH =  f"{DIALECT}://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/?service_name={SERVICE}" 

In [ ]:
engine = sqlalchemy.create_engine(ENGINE_PATH_WIN_AUTH)   

## Project Base Table
### Project
Project.agencyid = project sponsor

Implpaed = Implementing Agency for PA&ED

Implpse = Implementing Agency for PS&E

implcon = Implementing Agency for Construction

implrw = Implementing Agency for Right of Way


In [ ]:
projects_df = pd.read_sql_query(""" 
SELECT 
ctips_id,
appdate, 
archive,
agencyid,
const_date,
countyid,
countyid2,
countyid3,
chg_offcl,
chg_qual1,
chg_qual2,
districtid,
document,
docyear,
ea_number,
high_ver,
high_offcl,
implpaed, 
implpse, 
implrw, 
implcon, 
needpurpose,
progcode1,
ppno,
proj_desc,
postmiles1,
pm1b,
pm2b,
pm3b,
pm1a,
pm2a,
pm3a,
projcomp_date,
projectid,
route1,
route2,
route3,
rtl,
title,
version
FROM ctips.project
ORDER BY high_ver DESC, version DESC, high_offcl DESC
""", engine) 

In [ ]:
projects_table = _csis_utils.csis_clean_project(projects_df)

### PROJSCHE

In [ ]:
projsche_df = pd.read_sql_query(""" 
SELECT 
projectid,
m020 AS pa_ed_begin,
m200a AS pa_ed_end,
m200b AS ps_e_begin,
m224 AS begin_row,
m410 AS end_row,
m500 AS con_start_date,
m600 AS con_end_date,
m700 AS begin_closeout,
m800 AS end_closeout
FROM ctips.projsche
""", engine) 

In [ ]:
projsche_drop_cols = list(projsche_df.columns)

In [ ]:
projsche_drop_cols.remove('projectid')

In [ ]:
# I want to drop the rows in which ALL values in the date columns are empty
projsche_df2 = projsche_df.dropna(how = "all", subset = projsche_drop_cols).reset_index(drop = True)

In [ ]:
len(projsche_df2), len(projsche_df)

In [ ]:
projsche_df2.projectid.nunique()

In [ ]:
projsche_df2.projectid.value_counts().head()

#### Not a lot of matching values

In [ ]:
pd.merge(projsche_df2, projects_table, on ='projectid', how = 'outer', indicator = True)[['_merge']].value_counts()

In [ ]:
phase_dates_df = pd.merge(projects_table[['ctips_id', 'projectid']], projsche_df2,  on ='projectid', how = 'inner')

In [ ]:
phase_dates_df.ctips_id.nunique()

### AGENCY

In [ ]:
agency_df = pd.read_sql_query(""" 
SELECT 
name AS agency_name,
agencyid
FROM ctips.agncy
""", engine) 

In [ ]:
projects_table = pd.merge(projects_table, agency_df,  on ='agencyid', how = 'left').rename(columns = {'agency_name':'sponsoring_agency'})

In [ ]:
phase_agency_cols = ['implpaed','implpse','implrw', 'implcon']

In [ ]:
for i in phase_agency_cols:
    projects_table = _csis_utils.add_agencies(projects_table, agency_df, i)

In [ ]:
# Add spelled out version of the agency names to each column
projects_table = _csis_utils.add_agencies(projects_table, agency_df, 'implpse')

In [ ]:
projects_table = projects_table.drop(columns = phase_agency_cols)

### COUNTY

In [ ]:
county_df = pd.read_sql_query(""" 
SELECT 
name AS county_name,
countyid
FROM ctips.county
""", engine) 

In [ ]:
county_df.sample()

In [ ]:
county_df = pd.merge(projects_table[['ctips_id','countyid']], county_df,  on ='countyid', how = 'left')

In [ ]:
county_df = county_df[['ctips_id', 'county_name']]

### FUNDLINE
* For action: Action: P = programmed, V= vote, A=award

In [ ]:
fundline_df = pd.read_sql_query(""" 
SELECT 
action,
con,
rw,
pe_paed,
pe_env,
pe_rw,
pe_con,
pe_total,
fundlineid,
fundtypeid,
line_year,
actiondate
FROM ctips.fundline
""", engine) 

### Fundtype
* Fundtype.agencyid = funding agency

In [ ]:
fundtype_df = pd.read_sql_query(""" 
SELECT 
fundtypeid,
fundid,
progcode,
programid,
projectid,
agencyid
FROM ctips.fundtype
""", engine) 

#### Merge everything

In [ ]:
pd.merge(fundtype_df,
         fundline_df,  
         on = ['fundtypeid'], 
         how = "outer",
         indicator = True,)[['_merge']].value_counts()

In [ ]:
pd.merge(fundtype_df,fundline_df,  on = ['fundtypeid'], how = "outer", indicator = True)[['_merge']].value_counts()

In [ ]:
fund_m1 = pd.merge(fundtype_df,fundline_df,  on = ['fundtypeid'], how = "left")

In [ ]:
final_fund_m = pd.merge(projects_table[['projectid','ctips_id', 'document']], fund_m1, on = ['projectid'], how = "inner")

In [ ]:
final_fund_m.ctips_id.nunique(), len(final_fund_m)

In [ ]:
projects_table.ctips_id.nunique()

In [ ]:
29152-29116

### Progmain

In [ ]:
progmain_df = pd.read_sql_query(""" 
SELECT 
programid,
category AS program
FROM ctips.progmain
""", engine) 

In [ ]:
progmain_df.head()

### Fund

In [ ]:
fund_df = pd.read_sql_query(""" 
SELECT 
fund,
fundid,
type AS fund_type_1_fed_2_state_3_local
FROM ctips.fund
""", engine) 

In [ ]:
fund_df.head(2)

### Progsub
* Some progcodes have more than one progdesc
* Dropped duplicates bc the progdesc are similar
double_ids = ['20.30.010.820',
             '20.XX.723.000',
            '20.30.010.810',
             '20.XX.720.100',
             '20.30.010.817',
              '20.30.210.200'
             ]

In [ ]:
progsub_df = pd.read_sql_query(""" 
SELECT 
progcode,
progdesc
FROM ctips.progsub
""", engine) 

In [ ]:
progsub_df.head(1)

In [ ]:
# progsub_df.loc[progsub_df.progcode.isin(double_ids)].sort_values('progcode')

In [ ]:
progsub_df2 = progsub_df.drop_duplicates(subset = ['progcode'])

### Merge for work below

In [ ]:
final_fund_m.sample()

In [ ]:
final_fund_m.ctips_id.nunique(), projects_table.ctips_id.nunique()

In [ ]:
funding_w_program_info = ((final_fund_m
                           .merge(progmain_df, on = ['programid'], how = "left")
                           .merge(fund_df, on =['fundid'], how = "left")
                           .merge(progsub_df2, on = ['progcode'], how = "left"))
                           .drop(columns = ['fundid','programid', 'progcode']))

In [ ]:
 funding_w_program_info = funding_w_program_info.fillna(funding_w_program_info.dtypes.replace({'float64': 0.0, 'object': 'None'}))

In [ ]:
funding_w_program_info = funding_w_program_info

In [ ]:
funding_w_program_info.fund.nunique()

In [ ]:
funding_w_program_info['fund'] = funding_w_program_info.fund + '-' + funding_w_program_info.fund_type_1_fed_2_state_3_local.astype('str')

In [ ]:
funding_w_program_info = funding_w_program_info.fillna(funding_w_program_info.dtypes.replace({'float64': 0.0, 'object': 'None'}))

In [ ]:
funding_w_program_info.sample()

In [ ]:
funding_w_program_info.ctips_id.nunique()

## Phase Funding Table

#### First: find the # of funds a project has programmed/voted/awarded for each fund

In [ ]:
columns_to_agg = {**dict.fromkeys(['con', 'rw',
       'pe_paed', 'pe_env', 'pe_rw', 'pe_con', 'pe_total'], 'sum')}


In [ ]:
# I want to find the total funds a project will receive for each fund
total_cost = funding_w_program_info.groupby(['ctips_id','fund','document']).agg(columns_to_agg).reset_index()

In [ ]:
total_cost.sample()

##### Separate out FTIP and everything else to calculate total funds a project is estimated to receive

In [ ]:
# Calculate out FTIP and oither documents in 2 stages
ftip_only = total_cost.loc[total_cost.document == 'FTIP'].reset_index(drop = True)

In [ ]:
total_cost_ftip = ftip_only.groupby(['ctips_id', 'fund']).agg(columns_to_agg).reset_index()

In [ ]:
cols_to_keep = ['ctips_id', 'fund', 'total_cost']

In [ ]:
total_cost_ftip['total_cost'] = total_cost_ftip.con + total_cost_ftip.rw + total_cost_ftip.pe_total

In [ ]:
total_cost_ftip = total_cost_ftip[cols_to_keep]

In [ ]:
everything_else = total_cost.loc[total_cost.document != 'FTIP'].reset_index(drop = True)

In [ ]:
everything_else = everything_else.groupby(['ctips_id', 'fund']).agg(columns_to_agg).reset_index()

In [ ]:
everything_else['total_cost'] = everything_else.con + everything_else.rw + everything_else.pe_paed + everything_else.pe_env + everything_else.pe_rw + everything_else.pe_con

In [ ]:
everything_else = everything_else[cols_to_keep]

In [ ]:
total_requested_funds_final = pd.concat([everything_else, total_cost_ftip])

In [ ]:
len(total_requested_funds_final), total_requested_funds_final.ctips_id.nunique()

In [ ]:
total_requested_funds_final.ctips_id.value_counts().sample(5)

##### One project

In [ ]:
total_requested_funds_final.loc[total_requested_funds_final.ctips_id == 20300000209]

In [ ]:
8900.00+63400.00

In [ ]:
# Original 
funding_w_program_info.loc[funding_w_program_info.ctips_id == 20300000209]

#### Pivot - I want the dataframe to be wide instead of long

In [ ]:
fund_table = total_requested_funds_final.pivot_table(index=['ctips_id'], columns='fund', 
                    values=['total_cost'], aggfunc='sum')

In [ ]:
fund_table.columns = fund_table.columns.droplevel()

In [ ]:
fund_table = fund_table.reset_index()

In [ ]:
fund_table.shape

In [ ]:
fund_table = _utils.to_snakecase(fund_table)

##### Check with one project

In [ ]:
total_requested_funds_final.loc[total_requested_funds_final.ctips_id == 20300000522]

In [ ]:
fund_table.loc[fund_table.ctips_id == 20300000522].dropna(axis =1)

In [ ]:
total_requested_funds_final.loc[total_requested_funds_final.ctips_id == 20300000522]

In [ ]:
funding_w_program_info.loc[funding_w_program_info.ctips_id == 20300000522]

#### Second: find the amount of $ for each phase

In [ ]:
cost_per_phase = funding_w_program_info.groupby(['ctips_id']).agg(columns_to_agg).reset_index()

In [ ]:
cost_per_phase.shape, cost_per_phase.ctips_id.nunique()

In [ ]:
63400.00+9700.00+11300

In [ ]:
fund_table.loc[fund_table.ctips_id == 20300000209].dropna(axis=1)

In [ ]:
cost_per_phase.loc[cost_per_phase.ctips_id == 20300000209].dropna(axis=1)

#### Third: merge these 2 tables

In [ ]:
final_phase_funding_table = pd.merge(fund_table, cost_per_phase, on = ["ctips_id"], how = "inner")

In [ ]:
len(final_phase_funding_table)

In [ ]:
final_phase_funding_table.head(2)

In [ ]:
final_phase_funding_table.ctips_id.nunique(), len(final_phase_funding_table)

#### Fourth: find state v federal 
* State funds is a lot more.

In [ ]:

federal_funds = _csis_utils.calculate_state_fed_local_total_funds(final_phase_funding_table, ['1.0'], 'total_federal_funds')


In [ ]:
state_funds = _csis_utils.calculate_state_fed_local_total_funds(final_phase_funding_table, ['2.0'], 'total_state_funds')


In [ ]:
local_funds = _csis_utils.calculate_state_fed_local_total_funds(final_phase_funding_table, ['3.0'], 'total_local_funds')


In [ ]:
final_phase_funding_table["is_state"] = final_phase_funding_table.apply(_utils.is_state_funds, axis=1)

In [ ]:
final_phase_funding_table["is_federal"] = final_phase_funding_table.apply(_utils.is_fed_funds, axis=1)

In [ ]:
final_phase_funding_table["is_local"] = final_phase_funding_table.apply(_utils.is_local_funds, axis=1)

##### Check that I summed up federal funds correctly
* State not summing up correctly

In [ ]:
funding_w_program_info.loc[funding_w_program_info.ctips_id == 20920011849][['fund_type_1_fed_2_state_3_local']].value_counts()

In [ ]:
funding_w_program_info.loc[(funding_w_program_info.ctips_id == 20920011849) & (funding_w_program_info.fund == 'Public Transportation Modernization Improvement-2.0-2.0')][['con']].sum()

In [ ]:
funding_w_program_info.loc[(funding_w_program_info.ctips_id == 20920011849) & (funding_w_program_info.fund == 'Public Transportation Modernization Improvement-2.0-2.0')][['rw']].sum()

In [ ]:
funding_w_program_info.loc[(funding_w_program_info.ctips_id == 20920011849) & (funding_w_program_info.fund == 'Public Transportation Modernization Improvement-2.0-2.0')][['pe_total']].sum()

In [ ]:
112386000.00+12213000.00+4351000.00 == 128950000.0

In [ ]:
final_phase_funding_table.loc[final_phase_funding_table.ctips_id == 20920011849].dropna(axis=1).T

In [ ]:
(final_phase_funding_table
 .groupby(['is_state', 'is_federal', 'is_local'])
 .agg({'ctips_id':'nunique'})
 .reset_index()
 .sort_values(by = ['ctips_id']))

#### Fifth: Drop everything before `con`
* Need to differentiate between `pe_total` for FTIP vs `pe_total` for everything else.

In [ ]:
to_keep = ['ctips_id','con','rw', 'pe_env', 'pe_rw', 'pe_con', 'pe_total', 'total_federal_funds',
       'total_state_funds', 'total_local_funds', 'is_local', 'is_state',
       'is_federal']

In [ ]:
final_phase_funding_table2 = final_phase_funding_table[to_keep]

In [ ]:
final_phase_funding_table2.head()

#### Sixth: Merge on `phase_dates_df` with all the phase dates

In [ ]:
phase_dates_df.ctips_id.nunique(), len(phase_dates_df)

In [ ]:
projects_table.ctips_id.nunique(), len(projects_table)

In [ ]:
final_phase_funding_table.ctips_id.nunique(), len(final_phase_funding_table)

In [ ]:
final_phase_funding_table2 = pd.merge(final_phase_funding_table2, phase_dates_df, on = "ctips_id", how = "outer")

In [ ]:
pd.merge(final_phase_funding_table2, phase_dates_df, on = "ctips_id", how = "outer", indicator = True)[['_merge']].value_counts()

In [ ]:
final_phase_funding_table2.ctips_id.nunique()

In [ ]:
29152-29127

#### Seventh: Merge some other dates found in the `projects` dataframe.

In [ ]:
project_date_cols = ['const_date', 'rtl', 'ctips_id', 'projcomp_date']

In [ ]:
projects_dates = projects_table[project_date_cols]

In [ ]:
project_date_cols.remove('ctips_id')

In [ ]:
projects_table = projects_table.drop(columns = project_date_cols)

In [ ]:
# I'm only interested in rwos in which at least one of the dates are populated
projects_dates2 = projects_dates.loc[(projects_dates.rtl != 'datetime64[ns]') |  (projects_dates.const_date != 'datetime64[ns]')].reset_index(drop = True)

In [ ]:
projects_dates2 = projects_dates2.rename(columns = {'const_date': 'construction_completion_date', 'rtl':'ready_to_list_date'})

In [ ]:
final_phase_funding_table3 = pd.merge(final_phase_funding_table2, projects_dates2, on = 'ctips_id', how = 'left')

In [ ]:
final_phase_funding_table3.ctips_id.nunique()

In [ ]:
projects_table.ctips_id.nunique()

### Awards Table
* Take final_fund_m and sort it by year
* Line year is "fiscal year of this fund record Note that the year listed is the second in the pair of fiscal year notation.  For example if the funds for this record are for fiscal year 1998/99, then this record will hold the value 1999."
* These aren't really programs, funds?

In [ ]:
# del out '',
awards = funding_w_program_info[['ctips_id','line_year', 'program', 'progdesc']]

In [ ]:
# Just drop dups across
awards2 = awards.drop_duplicates().reset_index(drop = True)

In [ ]:
len(awards), len(awards2)

In [ ]:
awards3 = awards2.sort_values(by = ['ctips_id','program','line_year', ], ascending = [False, False, False])

In [ ]:
awards4 = awards3.drop_duplicates(subset = ['ctips_id','program'])

In [ ]:
awards4.ctips_id.nunique()

#### Check w/ one project

In [ ]:
awards4.loc[awards4.ctips_id == 20700001649]

In [ ]:
awards4.ctips_id.value_counts().describe()

In [ ]:
awards4.ctips_id.value_counts().head(10)

### Political

In [ ]:
political_df = pd.read_sql_query(""" 
SELECT 
*
FROM ctips.politcal
""", engine) 

In [ ]:
# Drop any rows with nulls
political_df = political_df.dropna(how = "any")

In [ ]:
len(political_df), political_df.projectid.nunique()

In [ ]:
political_df2 = pd.merge(projects_table[['ctips_id', 'projectid']], political_df, on ='projectid', how = 'inner')

In [ ]:
political_df2.shape, political_df2.projectid.nunique(), political_df2.ctips_id.nunique()

In [ ]:
assembly_df = _csis_utils.clean_political(political_df2, 'assembly')

In [ ]:
assembly_df.ctips_id.value_counts().head()

In [ ]:
len(assembly_df), assembly_df.ctips_id.nunique()

In [ ]:
senate_df = _csis_utils.clean_political(political_df2, 'ssenate')

In [ ]:
len(senate_df), senate_df.ctips_id.nunique()

In [ ]:
ushouse_df = _csis_utils.clean_political(political_df2, 'ushouse')

In [ ]:
len(ushouse_df), ushouse_df.ctips_id.nunique()

In [ ]:
ushouse_df.ushouse.value_counts().head()

#### Double check

In [ ]:
assembly_df.loc[assembly_df.ctips_id == 10900000289]

In [ ]:
political_df2.loc[political_df2.ctips_id == 10900000289]

In [ ]:
# projects_table.loc[projects_table.ctips_id == 10900000289]

## Save to Excel

In [ ]:
district_df = projects_table[['ctips_id','districtid']]

In [ ]:
district_df.shape

In [ ]:
drop_cols = ['chg_offcl', 'chg_qual1', 'chg_qual2','districtid', 'appdate', 'version','projcomp_date', 'agencyid', 'projectid', 'archive', 'agency_name']

In [ ]:
projects_table2 = projects_table.drop(columns = drop_cols)

In [ ]:
projects_table2 = projects_table2.fillna(projects_table2.dtypes.replace({'float64': 0.0, 'object': 'None'}))

In [ ]:
district_df = district_df.fillna(district_df.dtypes.replace({'float64': 0.0, 'object': 'None'}))

In [ ]:
county_df = county_df.fillna(county_df.dtypes.replace({'float64': 0.0, 'object': 'None'}))

In [ ]:
final_phase_funding_table3 = final_phase_funding_table3.fillna(district_df.dtypes.replace({'float64': 0.0, 'object': 'None'}))

In [ ]:
awards4 = awards4.fillna(awards4.dtypes.replace({'float64': 0.0, 'object': 'None'}))

In [ ]:
ushouse_df = ushouse_df.fillna(ushouse_df.dtypes.replace({'float64': 0.0, 'object': 'None'}))

In [ ]:
senate_df = senate_df.fillna(senate_df.dtypes.replace({'float64': 0.0, 'object': 'None'}))

In [ ]:
assembly_df = assembly_df.fillna(assembly_df.dtypes.replace({'float64': 0.0, 'object': 'None'}))

In [ ]:
assembly_df.ctips_id.nunique(), awards4.ctips_id.nunique()

In [ ]:
final_phase_funding_table3.ctips_id.nunique()

In [ ]:
county_df.ctips_id.nunique()

In [ ]:
district_df.ctips_id.nunique()

In [ ]:

with pd.ExcelWriter("./CTIPS.xlsx") as writer:
    projects_table2.to_excel(writer, sheet_name="project", index=False)
    district_df.to_excel(writer, sheet_name="district", index=False)
    county_df.to_excel(writer, sheet_name="county", index=False)
    final_phase_funding_table3.to_excel(writer, sheet_name="phase_funding", index=False)
    awards4.to_excel(writer, sheet_name="awards", index=False)
    ushouse_df.to_excel(writer, sheet_name="us_house", index=False)
    senate_df.to_excel(writer, sheet_name="senate", index=False)
    assembly_df.to_excel(writer, sheet_name="assembly", index=False)
